In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta

In [2]:
def get_current_datetime_info():
    # Get the current date and time
    current_datetime = datetime.now()

    # Extract components from the current date and time
    hour = current_datetime.hour
    day = current_datetime.day
    month = current_datetime.month
    year = current_datetime.year
    day_of_week = current_datetime.weekday()  # Monday is 0, Sunday is 6
    is_weekend = 1 if day_of_week >= 5 else 0  # 1 if Saturday (5) or Sunday (6), else 0

    # Create a DataFrame with the extracted information
    time_info = {'hour': [hour],
            'day': [day],
            'month': [month],
            'year': [year],
            'day_of_week': [day_of_week],
            'is_weekend': [is_weekend]}

    df = pd.DataFrame(time_info)
    return df

# Example usage
datetime_info_df = get_current_datetime_info()

In [3]:
def call_weather_api():
    # Get the current date and time
    current_datetime = datetime.now()

    # Subtract one hour to get the last completed hour
    previous_hour_datetime = current_datetime - timedelta(hours=1)

    # Format datetime to the required format "dd.MM.yyyy HH:00"
    formatted_date = previous_hour_datetime.strftime("%d.%m.%Y %H:00")

    # Construct the API URL with the formatted date
    api_url = f"https://luftdaten.berlin.de/api/stations/mc124/data?timespan=custom&start={formatted_date}&end={formatted_date}"

    # Make the API call
    response = requests.get(api_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(data)

        # Pivot the DataFrame so each unique core becomes a separate column
        if not df.empty:
            if 'core' in df.columns:
                df_pivot = df.pivot_table(index='datetime', columns='core', values='value')
                df_pivot.columns = [f'{col}' for col in df_pivot.columns]  # Rename columns for clarity
                df_pivot.reset_index(inplace=True)  # Optional: reset index to flatten the DataFrame
                return df_pivot
            else:
                print("The key 'core' is not found in the DataFrame.")
                return None
        else:
            print("The DataFrame is empty.")
            return None
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

data_df = call_weather_api()
if data_df is not None:
    print(data_df)


                    datetime  no_h-1  no2_h-1  nox_h-1  pm10_h-1  pm2_h-1
0  2024-09-12T19:00:00+02:00    38.0     50.0    109.0      15.0      7.0


TODO for the app
get current datetime and splitit into year, month, date, hour, day, is_weekend and remove timezone marker

create new df with current datetime and the values from the call_weather_api df and add _h-1 to each columnn except datetime